In [82]:
# from transformers import pipeline
# from text_rank.evaluation import *
# import torch
# 
# device = "cuda" if torch.cuda.is_available() else "cpu"
# summarizer = pipeline("summarization", model="KamilAin/bart-base-booksum", device=device)
# data = read_data("small_datasets/CNNML_tiny.csv")
# # Perform summarization

In [83]:
# summary = summarizer(str(data[0]), max_length=70, min_length=30, do_sample=False)
# # Print the summarized text
# print(summary[0]['summary_text'])

In [84]:
from transformers import BartTokenizerFast, BartForConditionalGeneration
from datasets import load_dataset

model_name = "lidiya/bart-base-samsum"

# Define the prefix and tokenizer
prefix = "summarize: "
tokenizer = BartTokenizerFast.from_pretrained(model_name)

# Function to preprocess and tokenize the dataset
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load the training dataset
train_dataset = load_dataset('csv', data_files="./tiny_CNN_DM/train_dataset.csv")
tokenized_train = train_dataset.map(preprocess_function, batched=True)

In [85]:
from transformers import Trainer, TrainingArguments
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BartForConditionalGeneration.from_pretrained(model_name)
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=60,
    weight_decay=1e-8,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=5
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"]
)

trainer.train()
model.save_pretrained("./bart_cnn_dailymail_finetuned")

Step,Training Loss


In [86]:
from transformers import BartTokenizerFast, BartForConditionalGeneration
from text_rank.evaluation import *
def generate_summary(text, model, tokenizer):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    inputs.to(device)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_new_tokens =50, max_length=120, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

model_path = "./bart_cnn_dailymail_finetuned"
tokenizer = BartTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_path)
model.to(device)
data = read_data("./tiny_CNN_DM/test_dataset.csv")
example_text = prefix + str(data[0])

summary = generate_summary(example_text, model, tokenizer)
print(summary)

Valdosta State University is in the center of a controversy after a video shared on social media showed an Air Force veteran and former Playboy model, Michelle Manhart, taking an American flag from demonstrators who had walked on it to protest racism. The protest, held a week ago, drew hundreds to the university. Protesters were trampling on the flag. Police said they traced the gun to a protester who was part of the flag-walking demonstration.


In [87]:
# from datasets import load_dataset, concatenate_datasets
# 
# def data_download():
#     dataset = load_dataset('kmyoo/cnn-dailymail-v1-tiny')
#     merged_dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])
#     merged_dataset.to_csv('./tiny_CNN_DM/full_dataset.csv')

In [88]:
# data_download()

In [89]:
# import pandas as pd
# 
# csv_file_path = './tiny_CNN_DM/full_dataset.csv'
# df = pd.read_csv(csv_file_path)

In [90]:
# from sklearn.model_selection import train_test_split
# 
# train_df, test_df = train_test_split(df, test_size=1/6)

In [91]:
# train_df.to_csv('./tiny_CNN_DM/train_dataset.csv', index=False)
# test_df.to_csv('./tiny_CNN_DM/test_dataset.csv', index=False)